# Run Summa model on HPC

### Retrieve a Summa model instance resource from HydroShare

#### For more info on this resource https://www.hydroshare.org/resource/1f3f310af8364d2aa3e6a9459152a21c/

In [ ]:
resource_id = '1f3f310af8364d2aa3e6a9459152a21c'
import json
import os
from hs_restclient import HydroShare
hs = HydroShare()
base_dir = os.path.abspath('/home/jovyan/work')
download_dir = os.path.join(base_dir, 'Downloads')
!mkdir -p {download_dir}
hs.getResource(resource_id, destination=download_dir, unzip=True)

### Unzip model instance package

In [ ]:
import os
#Unzip model file
model_folder_name = "SummaModel_ReynoldsAspenStand_StomatalResistance_sopron"
content_folder = os.path.join(download_dir ,"{}/{}/data/contents".format(resource_id, resource_id))
file_manger_rel_path = "settings/summa_fileManager_riparianAspenSimpleResistance.txt"
import tempfile
workspace_dir = os.path.join(base_dir, 'workspace')
!mkdir -p {workspace_dir}
unzip_dir = tempfile.mkdtemp(dir=workspace_dir)
!cd {content_folder} && unzip -o {model_folder_name}.zip -d {unzip_dir}
print("Unzipping Done")

### "Install" SUMMA model on Jupyter server (fix broken path in summa file_manager)

In [ ]:
model_source_folder_path = os.path.join(unzip_dir, model_folder_name)
!cd {model_source_folder_path} && chmod +x ./installTestCases_local.sh
!cd {model_source_folder_path} && ./installTestCases_local.sh

### Use PySumma to configure model parameters

In [ ]:
from pysumma import Simulation

# path to the SUMMA filemanager file on Jupyter
file_manager = os.path.join(model_source_folder_path, file_manger_rel_path)

# Create pySUMMA Simulation Object
executable = ""
S = Simulation(executable, file_manager)

# Configure the model
S.decisions['simulStart'].value = "2006-07-01 00:00"
S.decisions['simulFinsh'].value = "2007-08-20 00:00"
S.decisions['stomResist'].value = 'BallBerry'
# Save configiuration to disk
S._write_configuration()
print(S)

### Submit model to XSEDE Comet HPC using CyberGIS-Jupyter tool

In [ ]:
from cybergis import summaUI

s = summaUI(model_source_folder_path, file_manger_rel_path, 
            workspace_dir)

In [ ]:
s.runSumma()

### Visualization

In [ ]:
job_dir = os.path.join(workspace_dir, 
                            "{}/output".format(s.job_local_id))
summa_output = os.path.join(job_dir, 
                            "vegImpactsTranspire_output_test_timestep.nc")
print(summa_output)

### Check model output  -- NetCDF file

In [ ]:
import xarray as xr
output_info = xr.open_dataset(summa_output)
output_info

### Plot time series for total evapotranspiration

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
x = output_info['time']
y = (output_info['scalarCanopyTranspiration'] + \
     output_info['scalarCanopyEvaporation'] + \
     output_info['scalarGroundEvaporation'])*3600

plt.figure(figsize=(15, 5))
plt.plot(x, y, color='grey', linestyle='solid', markersize=0)

    # Get the current axis of the plot and
    # set the x and y-axis labels
ax = plt.gca()
ax.set_ylabel("Total ET (kg m-2 h-1)")
ax.set_xlabel('Date')
ax.grid(True)

ax.set_title('SUMMA Plot for Total ET')
output_info.close()

### Cleanup

In [ ]:
! rm -rvf {unzip_dir} {job_dir}

## Done